In [3]:
#pip install chess
#!pip install pgn2data
from converter.pgn_data import PGNData
import chess.pgn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# 1) Первичное преобразование файла

Как и планировалось, в основном этапе проекта, когда уже научились обращаться с небольшими файликами на 300к партий, мы берем более солидный датасет с партиями от мая 2017 года. В распакованном виде он весит 24 Гб и страшно представить, сколько он хранит партий (считать наверное не будем, и так циклов много будет)

1. **Вытаскиваем из этого большого файла фиксированное количество игр:**
Сначала была ппроделана попытка загнать в одну переменную 500 000 игр, однако, это оказалось крайне плохой идеей, так как уже на 370 000 компьютер начал зависать. Остановились на 300 000. 

    Вытащим их и положим в отдельный файл, а потом вытащим следующие 300 000 и положим в другой файл

In [3]:
def sometimes_i_pull_out_so_hard(pgnchik, n):
    games = []
    with open(pgnchik) as f:
        for _ in tqdm(range(n)):
            game = chess.pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games

pgnchik = "lichess_db_standard_rated_2017-05.pgn"
n = 300000

games = sometimes_i_pull_out_so_hard(pgnchik, n)

100%|██████████| 300000/300000 [16:03<00:00, 311.25it/s] 


2. **Записываем вытащенные игры в отдельный файл:**

In [4]:
def rip_the_skin(games, output_pgnchik):
    with open(output_pgnchik, "w") as f:
        for game in tqdm(games):
            pgn = chess.pgn.Game.from_board(game.board())
            pgn.headers.update(game.headers)
            f.write(str(pgn) + "\n\n")

output_pgnchik = "normas_games1.pgn"

rip_the_skin(games, output_pgnchik)

100%|██████████| 300000/300000 [01:10<00:00, 4281.73it/s]


3. **Проделаем операцию еще раз:**

    Для этого необходимо добавить всего одну строку в функцию

In [3]:
def sometimes_i_pull_out_so_hard(pgnchik, n):
    games = []
    with open(pgnchik) as f:
        for _ in range(n):
            chess.pgn.skip_game(f)
        for _ in tqdm(range(n)):
            game = chess.pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games

pgnchik = "lichess_db_standard_rated_2017-05.pgn"
n = 300000

games = sometimes_i_pull_out_so_hard(pgnchik, n)

100%|██████████| 300000/300000 [16:46<00:00, 298.04it/s] 


In [4]:
def rip_the_skin(games, output_pgnchik):
    with open(output_pgnchik, "w") as f:
        for game in tqdm(games):
            pgn = chess.pgn.Game.from_board(game.board())
            pgn.headers.update(game.headers)
            f.write(str(pgn) + "\n\n")

output_pgnchik = "normas_games2.pgn"

rip_the_skin(games, output_pgnchik)

100%|██████████| 300000/300000 [01:15<00:00, 3953.09it/s]


## Почему-то после применения данного кода из данных с играми не получилось скопировать ходы (файл moves оказался пустым). Применим другой код:
(Решили оставить данный раздел, потому что уж слишком гармонично и структурированно он выглядит)

4. **Используя новый код, вытащим из файла 200 000 игр и сразу запишем их в новый файл:**

In [22]:
def extract_games(input_pgn_file, output_pgn_file, num_games):
    # Открываем большой файл
    with open(input_pgn_file) as f:
        # Создаем маленький файл чтобы туда все записывать
        with open(output_pgn_file, 'w') as output_file:

            pgn = chess.pgn.read_game(f)

            # Заносим в переменную количество вытащенных игр
            games_extracted = 0
            while pgn is not None and games_extracted < num_games:
                # Записываем в файл
                output_file.write(str(pgn) + '\n\n')

                # Переходим к следующей игре
                pgn = chess.pgn.read_game(f)
                games_extracted += 1

                if games_extracted % 50000 == 0:
                    print(f"Extracted {games_extracted} games...")

input_pgn_file = 'lichess_db_standard_rated_2017-05.pgn'
output_pgn_file = 'massive_peacock.pgn'
num_games_to_extract = 200000

extract_games(input_pgn_file, output_pgn_file, num_games_to_extract)

Extracted 50000 games...
Extracted 100000 games...
Extracted 150000 games...
Extracted 200000 games...


4.1 **Игры вытащены, теперь преобразуем их в датафрейм, к которому будет впоследствии удобно присоединять дополнительные характеристики:**

In [23]:
#Перевод файла формата .pgn в .csv датасет
pgn_data = PGNData("massive_peacock.pgn")
result = pgn_data.export()
#в результате данная библиотека выводит два файла:
if result.is_complete:
    #файл с основным описанием игры
    games_df = result.get_games_df()
    print(games_df.head())
    #файл с ходами, сделанными в каждой игре, и некоторыми дополнительными характеристиками
    moves_df = result.get_moves_df()
    print(moves_df.head())

INFO:pgn2data - log_time:initializing at...2023-06-13 12:03:45.596857
INFO:pgn2data - pgn_data class:Starting process..
INFO:pgn2data - process:Processing file:massive_peacock.pgn
INFO:pgn2data - pgn_data class:ending process..
INFO:pgn2data - log_time:time taken sec: 7299.999002899999 sec
INFO:pgn2data - log_time:time taken: 2.0 hours, 1.0 minute, 39.99900289999914 seconds, 
INFO:pgn2data - log_time:time started...2023-06-13 12:03:45.596857
INFO:pgn2data - log_time:time ended.....2023-06-13 14:05:25.594919


                                game_id  game_order              event  \
0  7467c96c-2ffd-44c3-a15c-af055bed140b           1  Rated Bullet game   
1  9ab1f268-36e8-4ba2-98c1-9d681f5e622e           2  Rated Bullet game   
2  0f0dafc3-9ead-467c-a507-4028eba12c58           3  Rated Bullet game   
3  181a04ac-d94c-4ca4-bc0e-39516169367e           4  Rated Bullet game   
4  395f02d0-32f3-4e03-a23b-b03bdd5431e5           5   Rated Blitz game   

                           site date_played round       white          black  \
0  https://lichess.org/ObT3MGJ6  ????.??.??     ?  marshall91    Nechemevich   
1  https://lichess.org/wcL9cyTi  ????.??.??     ?    LTKAUNAS        Paupara   
2  https://lichess.org/D32us0RF  ????.??.??     ?     ateoluz   yigitmustafa   
3  https://lichess.org/6RBz5epC  ????.??.??     ?   le_batman  aleksandra_91   
4  https://lichess.org/z2b2Y95l  ????.??.??     ?    Trollson        salahi2   

  result  white_elo  ...  winner_loser_elo_diff  eco   termination  \
0   

In [21]:
moves_df.columns

Index(['game_id', 'move_no', 'move_no_pair', 'player', 'notation', 'move',
       'from_square', 'to_square', 'piece', 'color', 'fen', 'is_check',
       'is_check_mate', 'is_fifty_moves', 'is_fivefold_repetition',
       'is_game_over', 'is_insufficient_material', 'white_count',
       'black_count', 'white_pawn_count', 'black_pawn_count',
       'white_queen_count', 'black_queen_count', 'white_bishop_count',
       'black_bishop_count', 'white_knight_count', 'black_knight_count',
       'white_rook_count', 'black_rook_count', 'captured_score_for_white',
       'captured_score_for_black', 'fen_row1_white_count',
       'fen_row2_white_count', 'fen_row3_white_count', 'fen_row4_white_count',
       'fen_row5_white_count', 'fen_row6_white_count', 'fen_row7_white_count',
       'fen_row8_white_count', 'fen_row1_white_value', 'fen_row2_white_value',
       'fen_row3_white_value', 'fen_row4_white_value', 'fen_row5_white_value',
       'fen_row6_white_value', 'fen_row7_white_value', 'fen_row

In [24]:
games_df.loc[5,:]

game_id                  e42012cd-f4a2-4bf0-bfab-dd0e85f6fdb1
game_order                                                  6
event                                       Rated Bullet game
site                             https://lichess.org/d717OI8R
date_played                                        ????.??.??
round                                                       ?
white                                               kbknight1
black                                               cenotafio
result                                                    0-1
white_elo                                                1773
white_rating_diff                                       -11.0
black_elo                                                1780
black_rating_diff                                        10.0
white_title                                               NaN
black_title                                               NaN
winner                                              cenotafio
winner_e

5. **Все сработало. Вытаскиваем следующую партию игр (еще 200 000 штук):**

In [26]:
def extract_games(input_pgn_file, output_pgn_file, num_games):
    # Открываем большой файл
    with open(input_pgn_file) as f:
        # Создаем маленький файл чтобы туда все записывать
        with open(output_pgn_file, 'w') as output_file:
            for _ in range(num_games):
                chess.pgn.skip_game(f)

            pgn = chess.pgn.read_game(f)

            # Заносим в переменную количество вытащенных игр
            games_extracted = 0
            while pgn is not None and games_extracted < num_games:
                # Записываем в файл
                output_file.write(str(pgn) + '\n\n')

                # Переходим к следующей игре
                pgn = chess.pgn.read_game(f)
                games_extracted += 1

                if games_extracted % 50000 == 0:
                    print(f"Extracted {games_extracted} games...")

input_pgn_file = 'lichess_db_standard_rated_2017-05.pgn'
output_pgn_file = 'massive_fazan.pgn'
num_games_to_extract = 200000

extract_games(input_pgn_file, output_pgn_file, num_games_to_extract)

Extracted 50000 games...
Extracted 100000 games...
Extracted 150000 games...
Extracted 200000 games...


5.1 **Проделываем аналогичную операцию из пункта 4.1:**

In [3]:
#Перевод файла формата .pgn в .csv датасет
pgn_data1 = PGNData("massive_fazan.pgn")
result1 = pgn_data1.export()
#в результате данная библиотека выводит два файла:
if result1.is_complete:
    #файл с основным описанием игры
    games_df1 = result1.get_games_df()
    print(games_df1.head())
    #файл с ходами, сделанными в каждой игре, и некоторыми дополнительными характеристиками
    moves_df1 = result1.get_moves_df()
    print(moves_df1.head())

INFO:pgn2data - log_time:initializing at...2023-06-13 16:18:56.225428
INFO:pgn2data - pgn_data class:Starting process..
INFO:pgn2data - process:Processing file:massive_fazan.pgn
INFO:pgn2data - pgn_data class:ending process..
INFO:pgn2data - log_time:time taken sec: 5098.0804071 sec
INFO:pgn2data - log_time:time taken: 1.0 hour, 24.0 minutes, 58.08040710000023 seconds, 
INFO:pgn2data - log_time:time started...2023-06-13 16:18:56.225428
INFO:pgn2data - log_time:time ended.....2023-06-13 17:43:54.311936


                                game_id  game_order              event  \
0  66379645-adc7-4fed-8208-4561e010b813           1   Rated Blitz game   
1  bfa40e97-ed72-444b-a3b3-54ef803d8ac2           2  Rated Bullet game   
2  7f7f76bc-ab57-46db-b575-66f24b601217           3  Rated Bullet game   
3  963f8bc4-9ebe-42d7-b8e3-393d0bc9ae8f           4  Rated Bullet game   
4  f89b0c5e-576e-47a4-8f97-5bed4e0ec750           5  Rated Bullet game   

                           site date_played round       white  \
0  https://lichess.org/0EMWIB5D  ????.??.??     ?     Safiron   
1  https://lichess.org/56NnlbUW  ????.??.??     ?      AdiStf   
2  https://lichess.org/uylj2l8d  ????.??.??     ?  harshalp24   
3  https://lichess.org/T8t2RsGe  ????.??.??     ?  paerstymer   
4  https://lichess.org/gVN90Rj7  ????.??.??     ?  sekhar2204   

              black result  white_elo  ...  winner_loser_elo_diff  eco  \
0       AgentBroise    0-1       1343  ...                    -41  A00   
1        DIgorev

**Итог:**

    Получили 2 датасета в общей сумме на 400 000 партий. Раздельно проводить операции с ними будет в разы проще, однако придется часто перезапускать ядро. На первом этапе проекта был использовал 1 датасет с примерно 235 000 партиями и операции там проводились достатосно долго.
    
    Однако получившиеся итоговые преобразованные датасеты с необходимым набором данных содержали в себе всего около 2000 партий, что крайне плохо сказалось как на проверке гипотез, так и на машинном обучении. Именно поэтому сейчас было выбрано больше партий в надежде на то, что в итоговых данных останется хотя бы 10 0000.
    
    Первые 200 000 партий лежат в файлах "massive_peacock_moves.csv" и "massive_peacock_game_info.csv". Вторые 200 000 партий лежат в файлах "massive_fazan_moves.csv" и "massive_fazan_game_info.csv"

In [3]:
peacock_info = pd.read_csv("massive_peacock_game_info.csv")
peacock_moves = pd.read_csv("massive_peacock_moves.csv")

fazan_info = pd.read_csv("massive_fazan_game_info.csv")
fazan_moves = pd.read_csv("massive_fazan_moves.csv")

In [5]:
print(peacock_info.shape, fazan_info.shape)
print(peacock_moves.shape, fazan_moves.shape)

(200000, 29) (200000, 29)
(13466644, 64) (13544316, 64)


In [6]:
peacock_info.head()

,game_id,game_order,event,site,date_played,round,white,black,result,white_elo,...,winner_loser_elo_diff,eco,termination,time_control,utc_date,utc_time,variant,ply_count,date_created,file_name
0,7467c96c-2ffd-44c3-a15c-af055bed140b,1,Rated Bullet game,https://lichess.org/ObT3MGJ6,????.??.??,?,marshall91,Nechemevich,0-1,1926,...,43,B01,Time forfeit,60+0,2017.04.30,22:00:00,NaN,NaN,2023-06-13T12:03:45+0000,massive_peacock.pgn
1,9ab1f268-36e8-4ba2-98c1-9d681f5e622e,2,Rated Bullet game,https://lichess.org/wcL9cyTi,????.??.??,?,LTKAUNAS,Paupara,1-0,1592,...,165,A00,Time forfeit,60+0,2017.04.30,22:00:00,NaN,NaN,2023-06-13T12:03:45+0000,massive_peacock.pgn
2,0f0dafc3-9ead-467c-a507-4028eba12c58,3,Rated Bullet game,https://lichess.org/D32us0RF,????.??.??,?,ateoluz,yigitmustafa,1-0,1247,...,73,D00,Time forfeit,60+0,2017.04.30,22:00:00,NaN,NaN,2023-06-13T12:03:45+0000,massive_peacock.pgn
3,181a04ac-d94c-4ca4-bc0e-39516169367e,4,Rated Bullet game,https://lichess.org/6RBz5epC,????.??.??,?,le_batman,aleksandra_91,1-0,2047,...,38,A00,Time forfeit,60+0,2017.04.30,22:00:00,NaN,NaN,2023-06-13T12:03:45+0000,massive_peacock.pgn
4,395f02d0-32f3-4e03-a23b-b03bdd5431e5,5,Rated Blitz game,https://lichess.org/z2b2Y95l,????.??.??,?,Trollson,salahi2,1-0,1767,...,-14,B01,Time forfeit,300+0,2017.04.30,22:00:00,NaN,NaN,2023-06-13T12:03:45+0000,massive_peacock.pgn


In [7]:
fazan_info.head()

,game_id,game_order,event,site,date_played,round,white,black,result,white_elo,...,winner_loser_elo_diff,eco,termination,time_control,utc_date,utc_time,variant,ply_count,date_created,file_name
0,66379645-adc7-4fed-8208-4561e010b813,1,Rated Blitz game,https://lichess.org/0EMWIB5D,????.??.??,?,Safiron,AgentBroise,0-1,1343,...,-41,A00,Normal,240+3,2017.05.01,13:36:55,NaN,NaN,2023-06-13T16:18:56+0000,massive_fazan.pgn
1,bfa40e97-ed72-444b-a3b3-54ef803d8ac2,2,Rated Bullet game,https://lichess.org/56NnlbUW,????.??.??,?,AdiStf,DIgorevich,1-0,2089,...,42,A00,Normal,30+0,2017.05.01,13:36:56,NaN,NaN,2023-06-13T16:18:56+0000,massive_fazan.pgn
2,7f7f76bc-ab57-46db-b575-66f24b601217,3,Rated Bullet game,https://lichess.org/uylj2l8d,????.??.??,?,harshalp24,VLADIELIZAROVICH,1-0,1246,...,344,C00,Time forfeit,60+0,2017.05.01,13:36:56,NaN,NaN,2023-06-13T16:18:56+0000,massive_fazan.pgn
3,963f8bc4-9ebe-42d7-b8e3-393d0bc9ae8f,4,Rated Bullet game,https://lichess.org/T8t2RsGe,????.??.??,?,paerstymer,frajo,1-0,1726,...,-16,A43,Time forfeit,60+0,2017.05.01,13:36:56,NaN,NaN,2023-06-13T16:18:56+0000,massive_fazan.pgn
4,f89b0c5e-576e-47a4-8f97-5bed4e0ec750,5,Rated Bullet game,https://lichess.org/gVN90Rj7,????.??.??,?,sekhar2204,Anonim1976,0-1,1532,...,41,B20,Normal,60+0,2017.05.01,13:36:56,NaN,NaN,2023-06-13T16:18:56+0000,massive_fazan.pgn


**Все верно - в датафреймах различные игры, столбцы и содержание соответствует ожидаемому в обоих датафреймах**

# 2) Дополнительные характеристики

1. **Оценки ходов**

In [3]:
#создадим функцию для более удобного чтения исходных файлов игры
def read_pgn_file(file_path):
    with open(file_path) as f:
        game = chess.pgn.read_game(f)
        while game is not None:
            yield game
            game = chess.pgn.read_game(f)

Для первого датафрейма:

In [4]:
#Выведем некоторые признаки:
c1_list = [] #порядковый номер игры
c2_list = [] #ход в игре
c3_list = [] # оценка хода

for i, game in tqdm(enumerate(read_pgn_file('massive_peacock.pgn'))):
    numb = i + 1
    node = game
    while not node.is_end():
        next_node = node.variation(0)
        move = node.board().san(next_node.move)
        evaluation = next_node.eval()
        c1_list.append(numb)
        c2_list.append(move)
        c3_list.append(evaluation)
        node = next_node

200000it [1:26:30, 38.53it/s]


In [5]:
# Далее преобразуем листы в np.array и создадим соответсвующий датафрейм
c1 = np.array(c1_list)
c2 = np.array(c2_list)
c3 = np.array(c3_list)

new_data = pd.DataFrame()
new_data['Game'] = c1
new_data['Move'] = c2
new_data['Eval'] = c3

# И запишем результат в отдельный файл
#new_data.to_csv('evals_peackock.csv')

Для второго датафрейма:

In [ ]:
#Выведем некоторые признаки:
c1_list = [] #порядковый номер игры
c2_list = [] #ход в игре
c3_list = [] # оценка хода

for i, game in tqdm(enumerate(read_pgn_file('massive_fazan.pgn'))):
    numb = i + 1
    node = game
    while not node.is_end():
        next_node = node.variation(0)
        move = node.board().san(next_node.move)
        evaluation = next_node.eval()
        c1_list.append(numb)
        c2_list.append(move)
        c3_list.append(evaluation)
        node = next_node

In [ ]:
# Далее преобразуем листы в np.array и создадим соответсвующий датафрейм
c1 = np.array(c1_list)
c2 = np.array(c2_list)
c3 = np.array(c3_list)

new_data = pd.DataFrame()
new_data['Game'] = c1
new_data['Move'] = c2
new_data['Eval'] = c3

# И запишем результат в отдельный файл
#new_data.to_csv('evals_fazan.csv')

2. **Веремя ходов**

Для первого датафрейма:

In [9]:
c1_list = []  # Number of the game
c2_list = []  # Number of the move
c3_list = []  # Time for each move (%clk)

for i, game in tqdm(enumerate(read_pgn_file('massive_peacock.pgn'))):
    numb = i + 1
    node = game
    while not node.is_end():
        next_node = node.variation(0)
        move = node.board().san(next_node.move)
        comment = next_node.comment
        time_match = re.search(r"\%clk\s(\d+:\d+:\d+)", comment)
        time = time_match.group(1) if time_match else None
        c1_list.append(numb)
        c2_list.append(move)
        c3_list.append(time)
        node = next_node

200000it [1:28:02, 37.86it/s]


In [10]:
# Далее преобразуем листы в np.array и создадим соответсвующий датафрейм
c1 = np.array(c1_list)
c2 = np.array(c2_list)
c3 = np.array(c3_list)

new_data = pd.DataFrame()
new_data['Game'] = c1
new_data['Move'] = c2
new_data['Clock'] = c3

# И запишем результат в отдельный файл
#new_data.to_csv('clock_peackock.csv')

Для второго датафрейма:

In [ ]:
c1_list = []  # Number of the game
c2_list = []  # Number of the move
c3_list = []  # Time for each move (%clk)

for i, game in tqdm(enumerate(read_pgn_file('massive_fazan.pgn'))):
    numb = i + 1
    node = game
    while not node.is_end():
        next_node = node.variation(0)
        move = node.board().san(next_node.move)
        comment = next_node.comment
        time_match = re.search(r"\%clk\s(\d+:\d+:\d+)", comment)
        time = time_match.group(1) if time_match else None
        c1_list.append(numb)
        c2_list.append(move)
        c3_list.append(time)
        node = next_node

In [ ]:
# Далее преобразуем листы в np.array и создадим соответсвующий датафрейм
c1 = np.array(c1_list)
c2 = np.array(c2_list)
c3 = np.array(c3_list)

new_data = pd.DataFrame()
new_data['Game'] = c1
new_data['Move'] = c2
new_data['Clock'] = c3

# И запишем результат в отдельный файл
#new_data.to_csv('clock_fazan.csv')

3. **Дополнительное преобразование оценок ходов и объединение их с временем** 

Для первого датафрейма

In [5]:
clk = pd.read_csv('clock_peackock.csv')
evall = pd.read_csv('evals_peackock.csv')

In [28]:
#Соединяем
cleval = evall.join(clk[['Unnamed: 0', "Clock"]], on = 'Unnamed: 0', how = "left", lsuffix='_left', rsuffix='_right')
cleval = cleval.drop(columns = ['Unnamed: 0_left', 'Unnamed: 0_right', "Move"])

Для второго датафрейма

In [73]:
clk1 = pd.read_csv('clock_fazan.csv')
evall1 = pd.read_csv('evals_fazan.csv')

In [74]:
#Соединяем
cleval1 = evall1.join(clk1[['Unnamed: 0', "Clock"]], on = 'Unnamed: 0', how = "left", lsuffix='_left', rsuffix='_right')
cleval1 = cleval1.drop(columns = ['Unnamed: 0_left', 'Unnamed: 0_right', "Move"])

3. **Дебюты**

Для первого датафрейма

In [31]:
def read_pgn_file(file_path):
    with open(file_path) as f:
        game = chess.pgn.read_game(f)
        while game is not None:
            yield game
            game = chess.pgn.read_game(f)
# Извлечем из исходого датасета все дебюты, используя уже имеющуюся функцию чтения игр .pgn
openings = []
for i, game in tqdm(enumerate(read_pgn_file('massive_peacock.pgn'))):
        headers = game.headers
        for key, value in headers.items():
            if key == 'Opening':
                openings.append(value)

200000it [08:52, 375.76it/s]


In [32]:
op = np.array(openings)

Для второго датафрейма

In [76]:
openings1 = []
for i, game in tqdm(enumerate(read_pgn_file('massive_fazan.pgn'))):
        headers = game.headers
        for key, value in headers.items():
            if key == 'Opening':
                openings1.append(value)

200000it [08:16, 402.66it/s]


In [78]:
op1 = np.array(openings1)

# 4) Формирование финальных датасетов

1. **Разберемся с первым датасетом (peacock)**

Так как основным призаком, используещимся в данном проекте является оценка ходов игроков, в соотвествии с наличием подобных данных необходимо сократить имеющееся количество игр в датасете.

In [51]:
#Загрузим первый датасет еще раз:
peacock_info = pd.read_csv("massive_peacock_game_info.csv")
peacock_moves = pd.read_csv("massive_peacock_moves.csv")

In [52]:
# Добавим в датасат с ходами полученные численные оценки
peacock_moves['eval'] = cleval['Eval']
peacock_moves['clk'] = cleval['Clock']
# Оставим только те, для которых есть эти оценки
upd_peacock_moves = peacock_moves[peacock_moves['eval'].notnull()]
#Убедимся, что значения оценок, которые находились в формате PovScore превратятся в строки
upd_peacock_moves.loc[:, ['eval']] = upd_peacock_moves.loc[:, ['eval']].astype(str) #Оценки имеют игровой формат PovScore

C:\Users\79160\anaconda3\lib\site-packages\pandas\core\indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


* Дополнительное преобразование оценок ходов

In [53]:
# Получим список чисел,соответствующих оценку хода
numbers = []
strings = upd_peacock_moves['eval'].values
for string in tqdm(strings):
    matches = re.findall(r'-?\d+', string)
    for match in matches:
        numbers.append(int(match))
len(numbers), upd_peacock_moves['eval'].shape

100%|██████████| 1082773/1082773 [00:02<00:00, 469255.69it/s]


(1082773, (1082773,))

In [54]:
# Обновим значение колонки
upd_peacock_moves['eval'] = numbers

C:\Users\79160\AppData\Local\Temp/ipykernel_11000/3059804145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upd_peacock_moves['eval'] = numbers


### Теперь преобразуем датасет с общим описанием игр

In [55]:
#Дополнительно преобразуем колонку с типами игр
x = peacock_info['event'].to_frame()
x[['A', 'B']] = x['event'].str.split('https', 1, expand=True)
peacock_info['event'] = x['A']

#### Оставим только Rated Classical game, так как за короткое время в Blitz и Bullet трудно определить уровень игрока. Более того, в каждом из этих режимов разные стандарты рейтинга, что в разы усложнит дальнейшее машинное обучение

In [56]:
peacock_info['event'].value_counts()

Rated Blitz game                 79158
Rated Classical game             44959
Rated Bullet game                39561
Rated Blitz tournament           16464
Rated Bullet tournament          10716
Rated Classical tournament        5581
Rated UltraBullet tournament      1812
Rated UltraBullet game            1149
Rated Correspondence game          600
Name: event, dtype: int64

In [57]:
#Добавим дебюты
peacock_info['op'] = op
#Оставим только  Classical
peacock_info = peacock_info[peacock_info['event'] == 'Rated Classical game']

In [58]:
# Оставшиеся игры
games_left = upd_peacock_moves['game_id'].values
upd_peacock_info = peacock_info[peacock_info['game_id'].isin(games_left)]

In [59]:
# Оставим только интерпретируемые и некопирующиеся элементы
upd_peacock_info = upd_peacock_info[['game_id', 'event', 'white', 'black', 'result', 'white_elo',
       'black_elo', 'winner_loser_elo_diff', 'eco', 'termination', 'time_control', 'op']]

In [66]:
print(upd_peacock_info['game_id'].nunique())
print(upd_peacock_moves['game_id'].nunique())

5857
17065


In [68]:
#Необходимо подкорректировать датасет с ходами, так как мы укоротили количество игр засчет фиксации одного режима
upd_peacock_moves = upd_peacock_moves[upd_peacock_moves['game_id'].isin(upd_peacock_info['game_id'].values)]

Перед тем как загружать в отдельные файлы, проверим, все ли на месте:

In [69]:
print(upd_peacock_info.shape, upd_peacock_info.columns)
upd_peacock_info.head()

(5857, 12) Index(['game_id', 'event', 'white', 'black', 'result', 'white_elo',
       'black_elo', 'winner_loser_elo_diff', 'eco', 'termination',
       'time_control', 'op'],
      dtype='object')


,game_id,event,white,black,result,white_elo,black_elo,winner_loser_elo_diff,eco,termination,time_control,op
144,c17b10c9-84c5-4c85-8c3b-b7866d0e25de,Rated Classical game,TheYams,amir188,1-0,1607,1470,137,A04,Time forfeit,300+10,Zukertort Opening: Black Mustang Defense
184,63fbeb77-cbcf-4e0a-ba24-a9fcce5c5c4f,Rated Classical game,jarquincastillo,angelasturias,1-0,1723,1702,21,C34,Time forfeit,600+0,"King's Gambit Accepted, Fischer Defense"
209,6c678d21-9c2c-47cc-acc8-e4d4be1aeb51,Rated Classical game,Ygreek,roguemarvel,0-1,2030,1955,-75,B28,Normal,600+0,"Sicilian Defense: O'Kelly Variation, Normal Sy..."
224,77346e14-a662-424c-94f0-37330765c1d0,Rated Classical game,laserany,Theache,1-0,1462,1455,7,C61,Normal,600+0,Ruy Lopez: Bird Variation
292,ad728a3a-85df-4727-a596-e0c5e56b5f8e,Rated Classical game,minoulechat,Epicm14,0-1,1375,1142,-233,C42,Normal,480+0,Russian Game: Three Knights Game


In [70]:
print(upd_peacock_moves.shape, upd_peacock_moves.columns)
upd_peacock_moves.hefazanad()

(378926, 66) Index(['game_id', 'move_no', 'move_no_pair', 'player', 'notation', 'move',
       'from_square', 'to_square', 'piece', 'color', 'fen', 'is_check',
       'is_check_mate', 'is_fifty_moves', 'is_fivefold_repetition',
       'is_game_over', 'is_insufficient_material', 'white_count',
       'black_count', 'white_pawn_count', 'black_pawn_count',
       'white_queen_count', 'black_queen_count', 'white_bishop_count',
       'black_bishop_count', 'white_knight_count', 'black_knight_count',
       'white_rook_count', 'black_rook_count', 'captured_score_for_white',
       'captured_score_for_black', 'fen_row1_white_count',
       'fen_row2_white_count', 'fen_row3_white_count', 'fen_row4_white_count',
       'fen_row5_white_count', 'fen_row6_white_count', 'fen_row7_white_count',
       'fen_row8_white_count', 'fen_row1_white_value', 'fen_row2_white_value',
       'fen_row3_white_value', 'fen_row4_white_value', 'fen_row5_white_value',
       'fen_row6_white_value', 'fen_row7_white_val

,game_id,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,color,...,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence,eval,clk
10031,c17b10c9-84c5-4c85-8c3b-b7866d0e25de,1,1,TheYams,Nf3,g1f3,g1,f3,N,White,...,0,0,0,0,0,8,31,Nf3,-19,0:05:00
10032,c17b10c9-84c5-4c85-8c3b-b7866d0e25de,2,1,amir188,Nc6,b8c6,b8,c6,N,Black,...,0,0,0,0,3,8,28,Nf3|Nc6,42,0:05:00
10033,c17b10c9-84c5-4c85-8c3b-b7866d0e25de,3,2,TheYams,c4,c2c4,c2,c4,P,White,...,0,0,0,0,3,8,28,Nf3|Nc6|c4,-2,0:05:07
10034,c17b10c9-84c5-4c85-8c3b-b7866d0e25de,4,2,amir188,e5,e7e5,e7,e5,P,Black,...,0,0,0,1,3,7,28,Nf3|Nc6|c4|e5,10,0:04:55
10035,c17b10c9-84c5-4c85-8c3b-b7866d0e25de,5,3,TheYams,e3,e2e3,e2,e3,P,White,...,0,0,0,1,3,7,28,Nf3|Nc6|c4|e5|e3,4,0:05:06


In [71]:
print(upd_peacock_info['game_id'].nunique())
print(upd_peacock_moves['game_id'].nunique())

5857
5857


**Теперь все готово для финальной загрузки данных в удобные небольшие .csv файлы:**

In [72]:
#Запишем финальные датасеты в .csv файлы
upd_peacock_moves.to_csv('peacock_moves_df.csv')
upd_peacock_info.to_csv('peacock_info_df.csv')

2. **Проделаем все те же действия с вторым датасетом (fazan)**

In [79]:
fazan_info = pd.read_csv("massive_fazan_game_info.csv")
fazan_moves = pd.read_csv("massive_fazan_moves.csv")

In [80]:
# Добавим в датасат с ходами полученные численные оценки
fazan_moves['eval'] = cleval1['Eval']
fazan_moves['clk'] = cleval1['Clock']
# Оставим только те, для которых есть эти оценки
upd_fazan_moves = fazan_moves[fazan_moves['eval'].notnull()]
#Убедимся, что значения оценок, которые находились в формате PovScore превратятся в строки
upd_fazan_moves.loc[:, ['eval']] = upd_fazan_moves.loc[:, ['eval']].astype(str) #Оценки имеют игровой формат PovScore

C:\Users\79160\anaconda3\lib\site-packages\pandas\core\indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [82]:
# Получим список чисел,соответствующих оценку хода
numbers1 = []
strings = upd_fazan_moves['eval'].values
for string in tqdm(strings):
    matches = re.findall(r'-?\d+', string)
    for match in matches:
        numbers1.append(int(match))
len(numbers1), upd_fazan_moves['eval'].shape

100%|██████████| 1107283/1107283 [00:02<00:00, 500496.32it/s]


(1107283, (1107283,))

In [83]:
# Обновим значение колонки
upd_fazan_moves['eval'] = numbers1

C:\Users\79160\AppData\Local\Temp/ipykernel_11000/2908444629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upd_fazan_moves['eval'] = numbers1


In [84]:
#Дополнительно преобразуем колонку с типами игр
x = fazan_info['event'].to_frame()
x[['A', 'B']] = x['event'].str.split('https', 1, expand=True)
fazan_info['event'] = x['A']

In [85]:
fazan_info['event'].value_counts()

Rated Blitz game                 76905
Rated Classical game             43483
Rated Bullet game                39025
Rated Blitz tournament           16513
Rated Bullet tournament          13882
Rated Classical tournament        4952
Rated UltraBullet tournament      3497
Rated UltraBullet game            1028
Rated Correspondence game          715
Name: event, dtype: int64

In [86]:
#Добавим дебюты
fazan_info['op'] = op1
#Оставим только  Classical
fazan_info = fazan_info[fazan_info['event'] == 'Rated Classical game']

In [87]:
# Оставшиеся игры
games_left = upd_fazan_moves['game_id'].values
upd_fazan_info = fazan_info[fazan_info['game_id'].isin(games_left)]

In [91]:
# Оставим только интерпретируемые и некопирующиеся элементы
upd_fazan_info = upd_fazan_info[['game_id', 'event', 'white', 'black', 'result', 'white_elo',
       'black_elo', 'winner_loser_elo_diff', 'eco', 'termination', 'time_control', 'op']]

In [93]:
print(upd_fazan_info['game_id'].nunique())
print(upd_fazan_moves['game_id'].nunique())
#Необходимо подкорректировать датасет с ходами, так как мы укоротили количество игр засчет фиксации одного режима
upd_fazan_moves = upd_fazan_moves[upd_fazan_moves['game_id'].isin(upd_fazan_info['game_id'].values)]
print(upd_fazan_info['game_id'].nunique())
print(upd_fazan_moves['game_id'].nunique())

5574
5574
5574
5574


In [92]:
#Запишем финальные датасеты в .csv файлы
upd_fazan_moves.to_csv('fazan_moves_df.csv')
upd_fazan_info.to_csv('fazan_info_df.csv')

# 5) Объединение
Перед данным этапом была очищена память от всех переменных для наиболее оптимальной работы без зависаний

In [4]:
p_inf = pd.read_csv("peacock_info_df.csv")
p_mov = pd.read_csv("peacock_moves_df.csv")
f_inf = pd.read_csv("fazan_info_df.csv")
f_mov = pd.read_csv("fazan_moves_df.csv")

In [10]:
all_info = pd.concat([p_inf, f_inf],ignore_index=False)
all_moves = pd.concat([p_mov, f_mov],ignore_index=False)

**Замечание:** Такое большое количество промежуточных файлов делается на крайний случай, если вылетит Jupiter, неправильно перезапишутся переменнные или просто на каком-то моменте случится ошибка в коде и придется стирать все сохраненные перемнные. 

In [27]:
all_info.to_csv('all_info.csv')
all_moves.to_csv('all_moves.csv')

-----------

**Из файла, который весит 24 Гб., мы получили два датафрейма которые в сумме по весу равны примерно 300 Мб. Иногда задаешься вопросом, а ради чего все это..?**

Ответ на данный вопрос можно найти в следующем разделе проекта - "Visualisation and hypotheis testing"